In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
from src.assignment2.simulation import HospitalSimulation, PatientState, WardType, WardConfig, WardsConfigurations, HospitalConfiguration
from scipy import stats
import numpy as np


In [34]:

P = np.zeros([6,6])
P[0, :] = [0.0, 0.05, 0.10, 0.05, 0.80, 0.0]
P[1, :] = [0.20, 0.0, 0.50, 0.15, 0.15, 0.0]
P[2, :] = [0.30, 0.20, 0.0, 0.20, 0.30, 0.0]
P[3, :] = [0.35, 0.30, 0.05, 0.0, 0.30, 0.0]
P[4, :] = [0.20, 0.10, 0.60, 0.10, 0.0, 0.0] 
P[5, :] = [0.20, 0.20, 0.20, 0.20, 0.20, 0.0]

arr_Times = [14.5, 11.0, 8.0, 6.5, 5.0, 13.0]

len_stay = [2.9, 4.0, 4.5, 1.4, 3.9, 2.2]

urgency = [7, 5, 2, 10, 5, 0]

bed_capacity = [55, 40, 30, 20, 20, 0]


In [46]:

configs = [
    WardConfig(
        bed_capacity=55, 
        urgency=7, 
        mean_arrival_time=1/14.5, 
        mean_stay_time=2.9),
    WardConfig(40, 5, 1/11, 4),
    WardConfig(30, 2, 1/8, 4.5),
    WardConfig(20, 10, 1/6.5, 1.4),
    WardConfig(20, 5, 1/5, 3.9),
    WardConfig(0, 0, 1/13, 2.2)
]
std_configs = WardsConfigurations(configs)

std_switch = {
    WardType.A: [0.0, 0.05, 0.10, 0.05, 0.80, 0.0],
    WardType.B: [0.20, 0.0, 0.50, 0.15, 0.15, 0.0],
    WardType.C: [0.30, 0.20, 0.0, 0.20, 0.30, 0.0],
    WardType.D: [0.35, 0.30, 0.05, 0.0, 0.30, 0.0],
    WardType.E: [0.20, 0.10, 0.60, 0.10, 0.0, 0.0],
    WardType.F: [0.20, 0.20, 0.20, 0.20, 0.20, 0.0]
}

def mean_exp_dist(mean_time):
    return stats.expon.rvs(scale=mean_time)

sim_config = HospitalConfiguration(std_configs, std_switch, mean_exp_dist, mean_exp_dist)
hsim = HospitalSimulation(sim_config)

In [47]:
hsim.simulate_year()
hsim.total_penalty

hsim.wards[WardType.B]

Ward(type=<WardType.B: 1>, config=WardConfig(bed_capacity=40, urgency=5, mean_arrival_time=0.09090909090909091, mean_stay_time=4), total_number_of_treated_patients=3482, total_number_of_rejected_patients=0, patients=[(365.01604305021766, Patient(preferred_ward=<WardType.B: 1>, arrival_time=357.44391540929337, stay_time=7.572127640924266, state=<PatientState.IN_CORRECT_YARD: 1>, assigned_ward=<WardType.B: 1>)), (365.14186678778077, Patient(preferred_ward=<WardType.B: 1>, arrival_time=361.3746400262982, stay_time=3.767226761482571, state=<PatientState.IN_CORRECT_YARD: 1>, assigned_ward=<WardType.B: 1>)), (365.3867634161153, Patient(preferred_ward=<WardType.B: 1>, arrival_time=358.4967590072741, stay_time=6.890004408841205, state=<PatientState.IN_CORRECT_YARD: 1>, assigned_ward=<WardType.B: 1>)), (365.2359439956087, Patient(preferred_ward=<WardType.B: 1>, arrival_time=363.3413590224763, stay_time=1.8945849731324007, state=<PatientState.IN_CORRECT_YARD: 1>, assigned_ward=<WardType.B: 1>)),

In [23]:
## Example of setting bed distribution, individually and alltogether
hsim.ward_configs[WardType.A].bed_capacity = 40
hsim.ward_configs.bed_distribution = [100, 40, 10, 10, 10, 0]

## Example setting new distributions
def mean_lognorm_sd2(mean_time):
    s = 2/mean_time**2
    mu = mean_time * np.exp(-s**2/2)
    return stats.lognorm(scale=mu, s=np.sqrt(s)).rvs()

hsim.stay_dist = mean_lognorm_sd2


In [5]:

a = [(0.49527559250974806, Patient(type=PatientType.D, arrival_time=0.16369893942381536, stay_time=0.3315766530859327))]

In [34]:
hsim.wards[WardType.D]

Ward(type=<WardType.D: 3>, capacity=20, urgency=10, total_number_of_treated_patients=7, rejected_patients=0, patients=[(33.50380003701177, Patient(type=<PatientType.D: 3>, ward=<WardType.D: 3>, penalty=10, arrival_time=33.301781089755025, stay_time=0.20201894725674943))])